# Clustering Neghborhoods in Toronto

Using the Toronto neighborhood classification from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, we will explore the neighborhood using the Foursquare API and then create clusters using k-Means Clustering ALgorithm. Finally, we will map the clusters using Folium  

First,we import all the necessary libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import json 
import geocoder
from pandas import json_normalize
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Libraries imported.


Next, we use BeautifulSoup4 to parse the HTML script of the link above and find the table containing the pertinent information, i.e. the neighborhoods and postal codes. 

In [3]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(url.content,'html.parser')
interest = soup.find('table', class_='wikitable sortable')
rows = interest.find('tbody').find_all('tr')
rows_array=[] #to convert into ndarray
for x in rows:
    x_lst = [] #this would be a single row
    for y in x.find_all('td'):
        x_lst.append(y.get_text())
    x_lst = list(map(lambda s: s.strip(), x_lst)) #stripping the rows of whitespace characters that were carried over from the HTML script
    rows_array.append(x_lst)

We then map `rows_array` to a dataframe, `df`.

In [4]:
df = pd.DataFrame(rows_array)
df.rename(columns={0:'Postal Code', 1:'Borough', 2:'Neighborhood'}, inplace=True)
df.drop([0], inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
1,M1A,Not assigned,
2,M2A,Not assigned,
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


Getting the dimensions of the dataframe

In [6]:
df.shape

(180, 3)

Removing rows where Borough isn't assigned

In [7]:
df = df[df['Borough']!='Not assigned']
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


The new dimensions are now:

In [8]:
df.shape

(103, 3)

Next, we will use `geocoder` to get the coordinates of every borough. Since the package is unreliable and sometimes provides no results even though they exist, we will run a `while` loop till `geocoder` provides results we can store.

First we create columns for the latitude and longitude

In [9]:
df.loc[:, 'Latitude'] = np.nan
df.loc[:, 'Longitude'] = np.nan
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,NaN,NaN
3,M4A,North York,Victoria Village,NaN,NaN
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
5,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN


In [10]:
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

In [11]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN


In [12]:
for x in df['Postal Code']:
    pcindex = df.set_index('Postal Code').index.get_loc(x)
    coords = None
    while(coords is None):
        g = geocoder.bing('{}, Toronto, Ontario'.format(x), key='As1R7w1ShKBfPP0G1D40E18xHxVX_bgYkc6Asvvf2YpzNWxCRn5VtixACZwMnFTp')
        coords = g.latlng
    df.at[pcindex, 'Latitude'] = coords[0]
    df.at[pcindex, 'Longitude'] = coords[1]
    
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.751881,-79.330360
1,M4A,North York,Victoria Village,43.730419,-79.312820
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655140,-79.362648
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723209,-79.451408
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.664490,-79.393021
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653690,-79.511124
99,M4Y,Downtown Toronto,Church and Wellesley,43.666592,-79.381302
100,M7Y,East Toronto,Business reply mail Processing Centre,43.648689,-79.385437
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.632881,-79.489548
